In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import time

from ipywidgets import interact
from ipywidgets.widgets import IntSlider

Source: http://home.agh.edu.pl/~kzajac/dydakt/mownit/lab10/pde.pdf

Membrana w kształcie kwadratu o boku a jest równomiernie obciążona.
Membrana znajduje sie w stanie statycznym, a jej brzeg jest sztywno zamocowany i nieodkształcony.
Obliczyc odkształcenie membrany od poziomu , zakładając ze spełnia ono
równanie Poissona:

$$
\frac{\partial^2M}{\partial x^2} + \frac{\partial^2M}{\partial y^2} = \frac{-p}{T}
$$

Gdzie mamy ustalone parametry:
* $p>0$ - ciśnienie wywierane na membrane.
* $T$ - napięcie membrany.

$m$ - wartość napięcia membrany

Oznaczenia:
* $N$ := wymiary siatki
* $d = \frac{a}{N-1}$ := odległość między polami na siatce
* M := macierz $(N) \times (N)$ wartości napięcia membrany w danych punktach

$$
\frac{\partial^2 M}{\partial x^2} = \frac{M_{i+1,j} - 2 \cdot M_{ij} + M_{i-1,j} }{d^2}
$$

$$
\frac{\partial^2 M}{\partial y^2} = \frac{M_{i,j+1} - 2 \cdot M_{ij} + M_{i,j-1} }{d^2}
$$

Zapiszmy układ równań różniczkowych:

$$
\begin{cases}
    \forall i, j \in {1..N} : M_{1,j} = M_{N,j} = M_{i,1} = M_{i,N} = 0 \\
    M_{i+1,j}  + M_{i-1,j} + M_{i,j+1} + M_{i,j-1} - 4 \cdot M_{ij} = \dfrac{-p \cdot d^2}{T}
\end{cases}
$$

W celu zwektoryzowania siatki M, wprowadźmy wektor $\theta$:

$$
M_{ij} = \theta_{N\cdot i + j}
$$

Zauważmy, że dla danego $M{ij} = \theta_k$
$$
\begin{cases}
    M_{i-1, j} = \theta_{k - N} \\
    M_{i+1, j} = \theta_{k + N} \\
    M_{i, j-1} = \theta_{k-1} \\
    M_{i, j+1} = \theta_{k+1}
\end{cases}
$$

Po przekształceniu:

$$
\begin{cases}
    \forall i \in (1 \space .. \space N) : \theta_i = 0 
        \space\space\text{(krawędź górna)}\\
    \forall i \in (1 \space .. \space N) : \theta_{N(N-1) + i} = 0
        \space\space\text{(krawędź dolna)}\\
    \forall j \in (0 \space .. \space N-1) : \theta_{Nj + 1} = 0 
        \space\space\text{(krawędź lewa)}\\
    \forall j \in (1 \space .. \space N) : \theta_{Nj} = 0
        \space\space\text{(krawędź prawa)}\\
    \theta_{k-N}  + \theta_{k+N} + \theta_{k-1} + \theta_{k+1} - 4 \cdot \theta_{k} = \dfrac{-p \cdot d^2}{T}
        \space\space\text{(pozostałe punkty)}\\
\end{cases}
$$


In [ ]:
a = 1.0
p = 1.0
T = 1.0

N = 10
d = a / (N - 1)

a, p, T, N, d
NN = N ** 2 # number of datapoints

I'll denote indices of edges in order to manipulate them more easily

In [ ]:
def edges(N):
    NN = N ** 2
    up_edge = np.arange(0,N)
    down_edge = np.arange(NN - N, NN)
    left_edge = np.arange(N) * N
    right_edge = (np.arange(N) + 1) * N - 1
    return np.concatenate([up_edge, down_edge, left_edge, right_edge])

edges(N)

Right-hand vector of results

In [ ]:
def results(N, p, d, T):
    NN = N ** 2 
    result = np.array([(-p * d ** 2) / T for _ in range(NN)])
    result[edges(N)] = 0
    return result

results(N, p, d, T)

Matrix of coefficients:

In [ ]:
def matrix(N):
    edg = edges(N)
    NN = N ** 2
    coeff = np.eye(NN) * -4
    coeff[np.arange(1, NN), np.arange(0, NN - 1)] = 1 # elem to the left
    coeff[np.arange(0, NN - 1), np.arange(1, NN)] = 1 # elems to the right
    coeff[np.arange(N, NN), np.arange(0, NN - N)] = 1 # elems to the up
    coeff[np.arange(0, NN - N - 1), np.arange(N, NN - 1)] = 1 # elems to the down
    coeff[:, edg] = 0
    coeff[edg, edg] = 1
    return coeff

Thetas:

In [ ]:
def membrane(N, p, d, T):
    m = matrix(N)
    r = results(N, p, d, T)
    thetas = np.linalg.solve(m, r)
    return thetas.reshape(N,N)

In [ ]:
m = membrane(N, p, d, T)
sns.heatmap(m)

In [ ]:
df = pd.DataFrame(columns=['N', 'solution_time'])
Ns = np.arange(3, 104, 10)
trials = 10
Ns

In [ ]:
for N in Ns:
    for t in range(trials):
        t1 = time.time()
        membrane(N, p, d, T)
        t2 = time.time()
        t_d = t2 - t1
        df = df.append({
            'N': N,
            'solution_time': t_d
                       },
        ignore_index=True)    
        print('size: ' + str(N) + ', trial: ' + str(t) + ", time: " + str(t_d), end="\r")


In [ ]:
df_agg = df.groupby('N').mean()
df_agg

In [ ]:
plt.scatter(df_agg['solution_time'].keys(), df_agg['solution_time'])
plt.show()

In [ ]:
def disp_complexity(degree):
    X_b = df_agg['solution_time'].keys()
    Y_b = df_agg['solution_time']
    p = np.polyfit(X_b, Y_b, degree)
    X = np.array([X_b ** i for i in range(degree, -1, -1)]).T
    Y = X @ p
    print(p)
    print(((Y_b - Y) ** 2).sum())
    plt.scatter(X_b, Y_b)
    plt.plot(X_b, Y)
    plt.show()
    
interact(
    disp_complexity,
    degree=IntSlider(min=0, max=10)
)

Looks like $O(n^3)$ complexity. That's ugly!